# ML in Cybersecurity: Project II

## Team
  * **Team name**:  *fill this in*
  * **Members**:  *fill this in. format: name1 (email1), name2 (email2), ...*
  * **Tutor**: *fill this in*


## Logistics
  * **Due date**: 9th December 2018, 23:59:59 (to email the completed notebook to your respective tutors). We deduct 25% points for late submissions.
  * Complete this in the previously established **teams of 3**
  * Each team will be assigned a tutor. Mail the notebook (**do NOT share a link**) to the respective TAs before the due date.
  
  
## Timeline
  * 19-Nov-2018: Project 2 hand-out
  * **09-Dec-2018** (23:59:59): Email completed notebook to respective TAs
  * **10-Dec-2018** (12:15-14:00, regular tutorial slot): Teams present their projects
  * 17-Dec-2018 (12:15-14:00, regular tutorial slot): Project 2 discussion and summary
  
  
## About this Project
In this project, we dive into the vulnerabilities of machine learning models and the difficulties of defending them. To this end, we require you to implement an evasion attack (craft adversarial examples) yourselves, and defend your own model.   


## A Note on Grading
The total number of points in this project is 100. We further provide the number of points achievable with each excercise. You should take particular care to document and visualize your results, though.
 
We will evaluate this criteria in a presentation as for project 1. This will be during the regular tutorial slot on 10th December 12:00-14:00 (same location as last time). In case you have time restrictions during the time of the tutorial, notify us asap!  You can present on your laptops.
 
 
## Filling-in the Notebook
You'll be submitting this very notebook that is filled-in with (all!) your code and analysis. Make sure you submit one that has been previously executed in-order. (So that results/graphs are already visible upon opening it). 

The notebook you submit **should compile** (or should be self-contained and sufficiently commented). Check tutorial 1 on how to set up the Python3 environment.

It is extremely important that you **do not** re-order the existing sections. Apart from that, the code blocks that you need to fill-in are given by:
```
#
#
# ------- Your Code -------
#
#
```
Feel free to break this into multiple-cells. It's even better if you interleave explanations and code-blocks so that the entire notebook forms a readable "story".


## Code of Honor
We encourage discussing ideas and concepts with other students to help you learn and better understand the course content. However, the work you submit and present **must be original** and demonstrate your effort in solving the presented problems. **We will not tolerate** blatantly using existing solutions (such as from the internet), improper collaboration (e.g., sharing code or experimental data between groups) and plagiarism. If the honor code is not met, no points will be awarded.

 
 ## Versions
  * v1.0: Initial notebook
 
  ---

In [0]:
import time 
 
import numpy as np 
import matplotlib.pyplot as plt 

import json 
import time 
import pickle 
import sys 
import csv 
import os 
import os.path as osp 
import shutil 

import pandas as pd

from IPython.display import display, HTML
 
%matplotlib inline 
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots 
plt.rcParams['image.interpolation'] = 'nearest' 
plt.rcParams['image.cmap'] = 'gray' 
 
# for auto-reloading external modules 
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython 
%load_ext autoreload
%autoreload 2

In [0]:
# Load other libraries here.
# Keep it minimal! We should be easily able to reproduce your code.

# In case you want to use neural networks, we only support sklearn and keras (With a tensorflow backend).

#### Helpers

In case you choose to have some methods you plan to reuse during the notebook, define them here. This will avoid clutter and keep rest of the notebook succinct.

In [0]:
def identity_func(foo):
    return foo


#
#
# ------- Your Code -------
#
#

# 1. Attacking an ML-model

In this section, we implement an attack ourselves. We then leverage the Cleverhans library to craft adversarial examples. First, however, you need a model you can attack. Feel free to choose the DNN/ConvNN from your last project.

Hint: you might want to save the trained model to save time later.

### 1.1.1: Setting up the model (5 Points)

Re-use the model from your last project here and train it until it achieves reasonable accuracy (>92%).

In [0]:
#
#
# ------- Your Code -------
#
#

### 1.1.2: Implementing an attack (20 Points)

We now want you to attack the model trained in the previous step. You are free to choose any attack from the lecture, or implement your own idea. Take into account that the attacker is stealhy and wants to remain undetected: introduce a small change only! To test your attack, craft one adversarial example for the first occurence of each digit class in the training data.

In [ ]:
#
#
# ------- Your Code -------
#
#

### 1.1.3: Visualizing the results (5 Points)

Please visualize the ten adversarial examples you crafted in the previous step.

In [0]:
#
#
# ------- Your Code and plots-------
#
#

### 1.2.1: Using libraries for attacks (20 Points)
As the field of evasion attacks (in particular for DNN) is very active research field, several libraries have been published that contain attacks. We will work here with the CleverHans (https://github.com/tensorflow/cleverhans) library. Please implement FGSM and another attack of your choice using this library. Please plot for both attacks the resulting adversarial examples as before, craft however a bigger set of at least 1,000 examples using FGSM. 

In [0]:
#
#
# ------- Your Code -------
#
#

### 1.2.2: Visualizing the results (10 Points)
As before, please plot the results from the previous steps. Compare the results to your own attack. Which differences do you observe between the attacks? Why?

In [0]:
#
#
# ------- Your Code -------
#
#

# 2. Defending an ML model

So far, we have focused on attacking an ML model. In this section, we want you to defend your model. As before concerning the attack, you can chose an example from the lecture, or experiment with any idea you have.

We do not require the defense to work perfectly - but what we want you to understand is why it works or why it does not work.

### 2.1: Implementing a defense of your choice (25 Points)
As stated before, feel free to implement a defense or mitigation of your choice. Evaluate the defense on all adversarial examples you obtained from previous steps of this project and report the results.

In [0]:
#
#
# ------- Your Code -------
#
#

### 2.2: Conclusions (15 Points)
Please interpret the results of your defense here. What did you try to make the classifier more robust? Why did it work? Is the classifier now robust?

Feel free to state any interesting finding you encountered during this project.